In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

if not load_dotenv():
    raise FileNotFoundError("No .env file found")

Alpha Vantage is a API provider offering a large number of financial data operations.

https://www.alphavantage.co/documentation/

In [3]:
from llm_agents_introduction.alpha_vantage import AlphaVantageService


alpha_vantage = AlphaVantageService.create()

alpha_vantage.latest_price("MSFT")

453.55

In [4]:
from langchain_core.tools import tool


@tool
def get_stock_price(symbol: str):
    """Lookup the latest price of a stock by its symbol."""
    print(f"Looking up stock price of: {symbol}")
    return alpha_vantage.latest_price(symbol)


get_stock_price.invoke({"symbol": "MSFT"})

Looking up stock price of: MSFT


453.55

In [5]:
get_stock_price.name

'get_stock_price'

In [6]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant that helps the answer questions about the stock market using tools.",
        ),
        ("human", "{input_message}"),
    ]
)

tools = [get_stock_price]

gpt_4o = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

chain = prompt | gpt_4o.bind_tools(tools)

ai_message = chain.invoke({"input_message": "What is the current price of microsoft?"})

ai_message.tool_calls

[{'name': 'get_stock_price',
  'args': {'symbol': 'MSFT'},
  'id': 'call_ELkHyQ8Ofahw1XOgvIK7QZRF'}]

In [7]:
from langchain_core.tools import BaseTool
from langchain_core.messages.tool import ToolCall, ToolMessage


def find_tool(name: str) -> BaseTool:
    """Look up a set of tools by name"""
    tool = next(filter(lambda tool: tool.name == name, tools), None)

    if not tool:
        raise ValueError(f"No tool found with name: {name}")

    return tool


def call_tool(tool_call: ToolCall) -> ToolMessage:
    tool = find_tool(tool_call["name"])
    result = tool.invoke(tool_call["args"])


    return ToolMessage(tool_call_id=tool_call["id"], content=result)

For demonstration we build a primitive tool executor flow.
There are components in LangChain to do this all for you.

In [11]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts.chat import ChatPromptTemplate
from pprint import pprint


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant that helps the answer questions about the stock market using tools.",
        ),
        MessagesPlaceholder("message_history"),
    ]
)

chain = prompt | gpt_4o.bind_tools(tools)


message_history = []


def send_message(message: str):
    human_message = ("human", message)
    message_history.append(human_message)

    ai_message = chain.invoke({"message_history": message_history})

    message_history.append(ai_message)

    if ai_message.tool_calls:
        print('Received Tool Call:')
        pprint(ai_message.tool_calls[0])
        tool_message = call_tool(ai_message.tool_calls[0])
        message_history.append(tool_message)

    ai_message = chain.invoke(
        {
            "message_history": message_history,
        }
    )

    return ai_message


send_message("What's the current share price of microsoft?")

Received Tool Call:
{'args': {'symbol': 'MSFT'},
 'id': 'call_cCCA2M8bTXd9VzxkGM6ybJZw',
 'name': 'get_stock_price'}
Looking up stock price of: MSFT


AIMessage(content='The current share price of Microsoft (MSFT) is $453.55.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 101, 'total_tokens': 118}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_dd932ca5d1', 'finish_reason': 'stop', 'logprobs': None}, id='run-fadaac91-c6e6-4a18-8c6b-d2a312caff86-0', usage_metadata={'input_tokens': 101, 'output_tokens': 17, 'total_tokens': 118})

In [12]:
send_message("Is that more than Google?")

Received Tool Call:
{'args': {'symbol': 'GOOGL'},
 'id': 'call_BYs0vIBwJduw623ct4fuhAfa',
 'name': 'get_stock_price'}
Looking up stock price of: GOOGL


AIMessage(content="No, Microsoft's current share price of $453.55 is higher than Google's current share price of $185.07.", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0068660-2e40-4f27-96f2-8d7f033d7145-0', usage_metadata={'input_tokens': 140, 'output_tokens': 25, 'total_tokens': 165})

In [13]:
# new stock listings: https://stockanalysis.com/actions/listed/2024/

message_history = []
send_message("What's the current share price of 'Webtoon Entertainment Inc'?")

Received Tool Call:
{'args': {'symbol': 'WEBTOON'},
 'id': 'call_7MgoJo9q2iKHQDO3qWJsSht6',
 'name': 'get_stock_price'}
Looking up stock price of: WEBTOON


ValueError: Daily prices for not found for symbol: WEBTOON

In [51]:
import json
from llm_agents_introduction.alpha_vantage import SearchResult


@tool
def lookup_stock_symbol(company_name: str):
    """Search for stock symbols by a company name."""
    print(f"Searching for company: {company_name}")
    results: list[SearchResult] = alpha_vantage.search(company_name)

    content = [
        {"symbol": result.symbol, "company_name": result.name} for result in results
    ]
    print(f"Found results: {json.dumps(content)}")

    return json.dumps(content)


lookup_stock_symbol({"company_name": "Webtoon Entertainment Inc"})

Searching for company: Webtoon Entertainment Inc
Found results: [{"symbol": "WBTN", "company_name": "Webtoon Entertainment Inc"}]


'[{"symbol": "WBTN", "company_name": "Webtoon Entertainment Inc"}]'

In [52]:
tools = [get_stock_price, lookup_stock_symbol]

chain = prompt | gpt_4o.bind_tools(tools)


message_history = []
send_message("What's the current share price of 'Webtoon Entertainment Inc'?")

Searching for company: Webtoon Entertainment Inc
Found results: [{"symbol": "WBTN", "company_name": "Webtoon Entertainment Inc"}]


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rspXk54McyqPtVEyCrclgxTr', 'function': {'arguments': '{"symbol":"WBTN"}', 'name': 'get_stock_price'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 149, 'total_tokens': 165}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f139c680-3812-44aa-9dbe-b57feb9d0a33-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'WBTN'}, 'id': 'call_rspXk54McyqPtVEyCrclgxTr'}], usage_metadata={'input_tokens': 149, 'output_tokens': 16, 'total_tokens': 165})

In [53]:
def send_message(message: str):
    human_message = ("human", message)
    message_history.append(human_message)
    ai_message = None

    while not ai_message or ai_message.tool_calls:
        ai_message = chain.invoke({"message_history": message_history})

        message_history.append(ai_message)

        if ai_message.tool_calls:
            tool_message = call_tool(ai_message.tool_calls[0])
            message_history.append(tool_message)

    return ai_message

In [54]:
message_history = []
send_message("What's the current share price of 'Webtoon Entertainment Inc'?")

Searching for company: Webtoon Entertainment Inc
Found results: [{"symbol": "WBTN", "company_name": "Webtoon Entertainment Inc"}]
Looking up stock price of: WBTN


AIMessage(content='The current share price of Webtoon Entertainment Inc (symbol: WBTN) is $19.65.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 177, 'total_tokens': 199}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'stop', 'logprobs': None}, id='run-638bbd25-5cb6-4563-ba29-0ce11a58740f-0', usage_metadata={'input_tokens': 177, 'output_tokens': 22, 'total_tokens': 199})